In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import Bounds

In [2]:
tickers = ["QQQ", "VOO", "TSLA","MMM", "AAPL"]
start_date="2010-01-01"
end_date="2020-01-01"

#### ["Open","High","Low","Close","Adj Close","Volume"] for linear regression analysis and arbitrage use Open and Close, for correlations and sharpe ratio plus modern portfolio theory use Adj Close, also find mean median and mode of Volume

In [7]:
df = yf.download(tickers, start=start_date, end=end_date).loc[:, ["Open", "Close","Adj Close"]]
new_df = df.dropna()

[*********************100%***********************]  5 of 5 completed


In [9]:
new_df

Open                                                 \
                 AAPL         MMM         QQQ       TSLA         VOO   
Date                                                                   
2010-09-09   9.465714   83.919998   46.660000   1.400000  102.500000   
2010-09-10   9.399643   82.870003   46.480000   1.383333  101.680000   
2010-09-13   9.493571   84.559998   46.959999   1.392667  102.959999   
2010-09-14   9.507500   84.680000   47.189999   1.369333  102.839996   
2010-09-15   9.577500   84.470001   47.330002   1.398667  102.620003   
...               ...         ...         ...        ...         ...   
2019-12-24  71.172501  178.270004  212.000000  27.890667  295.359985   
2019-12-26  71.205002  176.449997  212.259995  28.527332  295.559998   
2019-12-27  72.779999  176.649994  214.539993  29.000000  297.510010   
2019-12-30  72.364998  177.259995  213.500000  28.586000  296.750000   
2019-12-31  72.482498  175.229996  211.529999  27.000000  294.529999   

                Close                                                 \
                 AAPL         MMM         QQQ       TSLA         VOO   
Date                                                                   
2010-09-09   9.395357   82.959999   46.430000   1.380667  101.320000   
2010-09-10   9.407500   83.940002   46.599998   1.344667  101.779999   
2010-09-13   9.537143   84.860001   47.250000   1.381333  103.059998   
2010-09-14   9.573571   84.629997   47.450001   1.408000  103.040001   
2010-09-15   9.650714   84.580002   47.750000   1.465333  103.300003   
...               ...         ...         ...        ...         ...   
2019-12-24  71.067497  176.679993  211.919998  28.350000  295.160004   
2019-12-26  72.477501  176.589996  213.789993  28.729334  296.670013   
2019-12-27  72.449997  177.259995  213.610001  28.691999  296.670013   
2019-12-30  72.879997  175.830002  212.210007  27.646667  295.040009   
2019-12-31  73.412498  176.419998  212.610001  27.888666  295.799988   

            Adj Close                                                 
                 AAPL         MMM         QQQ       TSLA         VOO  
Date                                                                  
2010-09-09   7.996561   57.755661   41.280602   1.380667   80.010040  
2010-09-10   8.006895   58.437935   41.431732   1.344667   80.373276  
2010-09-13   8.117236   59.078419   42.009644   1.381333   81.384048  
2010-09-14   8.148241   58.918282   42.187469   1.408000   81.368271  
2010-09-15   8.213901   58.883492   42.454189   1.465333   81.573601  
...               ...         ...         ...        ...         ...  
2019-12-24  69.517082  156.311813  208.175812  28.350000  281.141144  
2019-12-26  70.896332  156.232208  210.012772  28.729334  282.579437  
2019-12-27  70.869431  156.824982  209.835953  28.691999  282.579437  
2019-12-30  71.290047  155.559830  208.460693  27.646667  281.026855  
2019-12-31  71.810921  156.081787  208.853592  27.888666  281.750671  

[2344 rows x 15 columns]

In [16]:
new_df.loc[:,[("Open", "AAPL")]]

,Open
,AAPL
Date,
2010-09-09,9.465714
2010-09-10,9.399643
2010-09-13,9.493571
2010-09-14,9.507500
2010-09-15,9.577500
...,...
2019-12-24,71.172501
2019-12-26,71.205002


In [8]:
ret_df = np.log(new_df["Adj Close"]/new_df["Adj Close"].shift(1))
ret_df.corr("pearson")

,AAPL,MMM,QQQ,TSLA,VOO
AAPL,1.000000,0.394540,0.708322,0.252857,0.582348
MMM,0.394540,1.000000,0.647419,0.250586,0.738831
QQQ,0.708322,0.647419,1.000000,0.411655,0.923442
TSLA,0.252857,0.250586,0.411655,1.000000,0.370242
VOO,0.582348,0.738831,0.923442,0.370242,1.000000


In [ ]:
ret_df.cumsum().plot()

In [ ]:
ret_df.mean()

In [ ]:
ret_df.std()

In [ ]:
#equal,static weight portfolio
W = np.ones(len(ret_df.columns))/(np.ones(len(ret_df.columns))).sum()
W

In [ ]:
#Portfolio expected return
PEr = ret_df.mean().dot(W)
PEr

In [ ]:
#Portfolio Risk
PR = (W.T.dot(ret_df.cov().dot(W))) ** 0.5
PR

In [ ]:
#Assets Risk Adj. Return
ret_df.mean() / ret_df.std()

In [ ]:
#Portfolio Risk Adj. Return
RAr = PEr / PR
RAr

In [ ]:
#Weight optimization
def sharpe_pf_ratio(W, returns):
    PR = (W.T.dot(ret_df.cov().dot(W))) ** 0.5
    SR = W.dot(returns.mean()) / PR
    return -SR

In [ ]:
#sharpe ratio of equal w8 portfolio
sharpe_pf_ratio(W, ret_df)

In [ ]:
#constraints
cons = ({"type":"eq", "fun": lambda x: np.sum(x) - 1})

In [ ]:
results = minimize(sharpe_pf_ratio, W, ret_df, bounds=Bounds(0,1), constraints=cons)

In [ ]:
opt_W = results["x"]
opt_W 

In [ ]:
#split train test data
train = ret_df[:int(len(ret_df) * 0.7)]
test = ret_df[int(len(ret_df) * 0.7):]

In [ ]:
res_train = minimize(sharpe_pf_ratio, W, train, bounds=Bounds(0,1), constraints=cons)
res_train

In [ ]:
#optimized weights
test.dot(res_train["x"]).cumsum()

In [ ]:
#equal weights
test.dot(W).cumsum()

### The smaller the sharp ratio the more performant the better the portfolio

In [ ]:
sharpe_pf_ratio(res_train["x"], test)

In [ ]:
sharpe_pf_ratio(W, test)